In [1]:
'''
Coding: PYTHON UTF-8
Created On: 2023-04-28 12:39:30
Author: Putu Hendra Widyadharma
=== filter .pha using wadati result from makeWADATI 
'''

'\nCoding: PYTHON UTF-8\nCreated On: 2023-04-28 12:39:30\nAuthor: Putu Hendra Widyadharma\n=== filter .pha using wadati result from makeWADATI \n'

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from localfunction import *
pd.options.mode.chained_assignment = None

In [3]:
path = 'G:\\My Drive\\Tomography\\280423\\output-tomodd-relocvelest-damp130-28042023\\'
fname = 'phase_sul_2022_8P_fix_sta-rms5.dat'
fname2 = 'temppha_phase_sul_2022_8P_fix_sta-rms5.txt'
fname3 = 'tempwadati_phase_sul_2022_8P_fix_sta-rms5.txt'
deviation = 9.5

In [4]:
df = readabsolute(path+fname)
dfphase = pd.read_csv(path+fname2,delim_whitespace=True)
dfwadati = pd.read_csv(path+fname3,delim_whitespace=True)

In [5]:
# hitung dan buat persamaan pembatas 
slope, intercept = np.polyfit(dfwadati['tp'].astype(float),dfwadati['ts-tp'].astype(float),1)

# predict function
predict = np.poly1d([slope, intercept])
# predict with deviation
predictup = np.poly1d([slope,intercept+deviation])
predictdown = np.poly1d([slope,intercept-deviation])

# hitung R^2 menguji kecocokan regresi
r2 = r2_score(dfwadati['ts-tp'],predict(dfwadati['tp']))
print('VP/VS = ' + str(slope + 1))
print(f"R-square = {r2}")

VP/VS = 1.7653324367545147
R-square = 0.9755771631288414


In [6]:
### dfwadati processing
# fungsi seleksi data berdasarkan batas atas dan bawah deviasi
def selection_data(row):
    if (row['ts-tp'] <= predictup(row['tp'])) and (row['ts-tp'] >= predictdown(row['tp'])):
        return True
    else:
        return False
# apply perhitungan
dfwadati['result'] = dfwadati.apply(selection_data,axis=1)
# select wadati hanya pada event yang perlu diubah
dfwadati = dfwadati[dfwadati['event_id'].isin(dfwadati[dfwadati['result'] == False]['event_id'].unique())]

In [7]:
# index search untuk event_id yang perlu disesuaikan menurut wadati 
editable_idx = df[df[0] == '#'][14].astype('int64').isin(dfwadati['event_id'].unique())
editable_idx = editable_idx[editable_idx==True].index
editable_idx

Int64Index([ 17670,  39855,  42761,  49187,  49780,  54517,  56727,  61629,
             62294,  63963,  65792,  65832,  66268,  76167,  85090,  93991,
             94923,  96299,  97449, 100481, 102172, 103661, 105042, 109854,
            114636, 115814, 120764, 124919, 124955, 126667, 130394, 131327,
            137130],
           dtype='int64')

In [8]:
# buat tempat data hasil
df_result = pd.DataFrame([],columns = df.columns)

# pha index iterable
idx = df[df[0] == '#'].index

for idx_ in range (len(idx)):
# for idx_ in [1099]:
# for idx_ in [0,1,2,3]:
    # buat data per kejadian gempa
    if idx_ == len(idx)-1:
        tempdf = df.iloc[idx[idx_]::]
    else:
        tempdf = df.iloc[idx[idx_]:idx[idx_+1],:]
    # cek index header terhadap editable_idx
    if tempdf.index[0] in editable_idx:
        # proses delete data fasa S stasiun yang ts-tp diluar batas deviasi
        for i, row_ in tempdf.iterrows():
            # cari fasa S dan list stasiun yang masuk dalam membuat hasil wadati buruk 
            if row_[3]=='S' and row_[0] in (dfwadati[(dfwadati['event_id'] == int(tempdf.iloc[0,14])) & (dfwadati['result'] == False)]['sta'].to_list()):
                print(f'event: {tempdf.iloc[0,14]} -- index:{row_.name} --> {row_[0]} Phase:{row_[3]} dropped')
                tempdf.drop(row_.name,inplace=True)
    else:
        pass
    # masukkan hasil ke df hasil
    df_result = pd.concat([df_result,tempdf])

event: 1100 -- index:17672 --> TMSI Phase:S dropped
event: 2628 -- index:39870 --> GTOI Phase:S dropped
event: 2869 -- index:42768 --> SGSI Phase:S dropped
event: 3449 -- index:49198 --> SPSI Phase:S dropped
event: 3501 -- index:49788 --> SPSI Phase:S dropped
event: 3813 -- index:54524 --> PMSI Phase:S dropped
event: 3938 -- index:56729 --> TNTI Phase:S dropped
event: 4207 -- index:61633 --> KMSI Phase:S dropped
event: 4207 -- index:61638 --> SRSI Phase:S dropped
event: 4207 -- index:61640 --> TOLI2 Phase:S dropped
event: 4240 -- index:62298 --> MRSI Phase:S dropped
event: 4240 -- index:62300 --> LUWI Phase:S dropped
event: 4328 -- index:63967 --> TMSI Phase:S dropped
event: 4423 -- index:65800 --> PSSI Phase:S dropped
event: 4426 -- index:65834 --> KMSI Phase:S dropped
event: 4444 -- index:66286 --> LUWI Phase:S dropped
event: 5256 -- index:76170 --> MPSI Phase:S dropped
event: 5256 -- index:76172 --> LOCM Phase:S dropped
event: 6006 -- index:85118 --> SDCI Phase:S dropped
event: 6834

In [9]:
# perbaiki df_result dan simpan dalam bentuk pha
df_result.sort_index(inplace = True)
df_result.reset_index(inplace = True, drop = True)
df2dat(df_result,evnum = 1, path = path, fname=Path(fname).stem+'_wadatifilter.dat')
print("== data filter")
readeventphase(path+Path(fname).stem+'_wadatifilter.dat')

Output finish: phase_sul_2022_8P_fix_sta-rms5_wadatifilter.dat at G:\My Drive\Tomography\280423\output-tomodd-relocvelest-damp130-28042023\
== data filter
! phase_sul_2022_8P_fix_sta-rms5_wadatifilter.dat
---------------------------
start time        : 2017-01-02 17:16:39.400000
end time          : 2022-12-31 22:21:43.900000
minimum magnitude : 2.5
maximum magnitude : 7.39
minimum depth     : 1 km
maximum depth     : 750 km
station used      : 128
---------------------------
Event :  10421
Phase : 127774
-- P  : 103494
-- S  :  24101
-- Pg :    103
-- Sg :     60
-- Pn :     15
-- pP :      1
---------------------------
